In [1]:

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import re

nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack

pd.set_option('display.max_columns', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\morga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\morga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [50]:
# df = pd.read_parquet("machine_learning_v2.parquet")

In [27]:
# df[df.isna().any(axis=1)]
# df.dropna(inplace=True)

In [51]:
# df

,titre_id,titre_str,titre_genres,rating_avg,rating_votes,actors,directors,overview
0,tt0035423,Kate & Leopold,"[Comedy, Fantasy, Romance]",6.4,87897,"[Meg Ryan, Hugh Jackman, Liev Schreiber, Breck...",[James Mangold],When her scientist ex-boyfriend discovers a po...
1,tt0052832,The Fugitive Kind,"[Drama, Romance]",7.1,6914,"[Anna Magnani, Marlon Brando, Joanne Woodward,...",[Sidney Lumet],"Val Xavier, a drifter of obscure origins, arri..."
2,tt0052997,The League of Gentlemen,"[Comedy, Crime, Drama]",7.2,5105,"[Jack Hawkins, Roger Livesey, Nigel Patrick, R...",[Basil Dearden],Involuntarily-retired Colonel Hyde recruits se...
3,tt0053459,Eyes Without a Face,"[Drama, Horror]",7.6,34278,"[Pierre Brasseur, Alexandre Rignault, Alida Va...",[Georges Franju],Dr. Génessier is riddled with guilt after an a...
4,tt0053579,Late Autumn,"[Comedy, Drama]",7.9,5854,"[Setsuko Hara, Keiji Sada, Mariko Okada, Yôko ...",[Yasujirô Ozu],A woman and her daughter are each forced to co...
...,...,...,...,...,...,...,...,...
7411,tt9873892,They Cloned Tyrone,"[Comedy, Mystery, Sci-Fi]",6.6,33197,"[Kiefer Sutherland, Jamie Foxx, John Boyega, T...",[Juel Taylor],A series of eerie events thrusts an unlikely t...
7412,tt9883996,Dream Horse,"[Biography, Comedy, Drama]",6.9,6589,"[Alan David, Lynda Baron, Owen Teale, Toni Col...",[Euros Lyn],"The inspiring true story of Dream Alliance, an..."
7413,tt9893250,I Care a Lot,"[Comedy, Crime, Drama]",6.4,141372,"[Dianne Wiest, Rosamund Pike, Peter Dinklage, ...",[J Blakeson],A court-appointed legal guardian defrauds her ...
7414,tt9900782,Kaithi,"[Action, Adventure, Crime]",8.5,39546,"[Narain, Karthi, George Maryan, Arjun Das]",[Lokesh Kanagaraj],"Dilli, a convicted criminal, is out on parole ..."


In [52]:
# tt = (
#     ("actors", "actors"),
#     ("titre_genres", "titre_genres"),
#     ("directors", "directors"),
# )
# for t in tt:
#     df[t[0]] = df[t[1]].apply(lambda x: ", ".join(map(str, x))).replace(" ", "")

In [63]:
# def clean_overview(
#     text: str
# ):
#     text = text.lower()
#     text = re.sub(r'[^a-z]', ' ', text)
#     words = text.split()
#     words = [word for word in words if word not in stopwords.words('english')]
#     lemmatizer = WordNetLemmatizer()
#     words = [lemmatizer.lemmatize(word) for word in words]
#     return ' '.join(words)

# df['overview'] = df['overview'].astype(str).apply(clean_overview)

In [314]:
# df.to_parquet("machine_learning_nltk_list.parquet")
df = pd.read_parquet("machine_learning_nltk_list.parquet")

In [315]:
def full_lower(
    text: str
):
    return text.replace(" ", "").replace("-", "")

tt = (
    ("actors", "actors"),
    ("titre_genres", "titre_genres"),
    ("directors", "directors"),
)
for t in tt:
    df[t[0]] = df[t[1]].apply(full_lower)


# df["overview"] = df["overview"].apply(lambda x : x.replace(" ", ","))


In [316]:
print(df[df["titre_str"].str.contains("Fight Club")].to_markdown())


|      | titre_id   | titre_str   | titre_genres   |   rating_avg |   rating_votes | actors                                     | directors    | overview                                                                                                                                                                                                                     |
|-----:|:-----------|:------------|:---------------|-------------:|---------------:|:-------------------------------------------|:-------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 2689 | tt0137523  | Fight Club  | Drama          |          8.8 |        2250929 | MeatLoaf,ZachGrenier,BradPitt,EdwardNorton | DavidFincher | ticking time bomb insomniac slippery soap salesman channel primal male aggression shocking new form therapy concept cat

In [317]:
print(df[df["titre_id"].str.contains("tt1023111")].to_markdown())


|      | titre_id   | titre_str       | titre_genres       |   rating_avg |   rating_votes | actors                                         | directors   | overview                                                                                                                                                                                                                                                                                                             |
|-----:|:-----------|:----------------|:-------------------|-------------:|---------------:|:-----------------------------------------------|:------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 4377 | tt1023111  | Never Back Down | Action,Drama,Sport |      

In [318]:
movie = "The Avengers"

In [280]:
# feat = [
#     'titre_str',
#     'titre_genres',
#     # 'rating_avg',
#     # 'rating_votes',
#     "actors",
#     'directors',
# ]
# # df.columns.to_list()
# feat

# for f in feat:
#     df[f] = df[f].replace(" ", "")

In [319]:
def title_index(titre):
    return df[df.titre_str == titre].index[0]

def idx_title(idx):
    return df[df.index == idx]["titre_str"].values[0]

In [320]:
def combine(r):
    return r["titre_genres"]+" "+r["directors"]+" "+r["overview"]+" "+r["actors"] #r["titre_str"]+" "+

df["one_for_all"] = df.apply(combine, axis=1)

In [284]:
def choice(movie: str):
    if movie == "genres":
        poids_ = {
            # "titre_str":    0.2,
            "titre_genres": 0.2,
            "actors":       0.2,
            "directors":    0.2,
            "overview":     0.2,
        }


In [331]:
# tidf = TfidfVectorizer()
# # combined_matrix = tidf.fit_transform(
# #     df["titre_str"]+" "+df["actors"]+" "+df["titre_genres"]+" "+df["directors"]+" "+df["overview"]
# # )
# combined_matrix = tidf.fit_transform(
#     df["one_for_all"]+" "+df["titre_str"]
# )
poids_ = {
    # "titre_str":    0.2,
    "titre_genres": 0.2,
    "actors":       0.2,
    "directors":    0.2,
    "overview":     0.5,
    # "rating_avg":   0.2,
    # "rating_votes": 0.2,
}

top = 5

full_matrix = []
for col, poids in poids_.items():
    tfidf_ = TfidfVectorizer()
    matrix_ = tfidf_.fit_transform(df[col]) * poids
    full_matrix.append(matrix_)
combined_matrix = hstack(full_matrix)
cosine = cosine_similarity(combined_matrix)
movies = "Fight Club"
mov_idx = title_index(movies)
similar = cosine[mov_idx]
similar1 = list(enumerate(cosine[mov_idx]))
sim_scores = sorted(similar1, key=lambda x: x[1], reverse=True)
sim_mov_idx = similar.argsort()[::-1][1:top+1]
same_movies = df.loc[sim_mov_idx, "titre_str"]
ttconst = df.loc[sim_mov_idx, "titre_id"]
sim_scores[1:top+1]
score = [i[1] for i in sim_scores]
for movies, tt,  score in zip(same_movies, ttconst, score):
    print(f"Movie : {movies} | id : {tt} | score : {np.round(score, 4)}")


Movie : After the Wedding | id : tt7985692 | score : 1.0
Movie : Exotica | id : tt0109759 | score : 0.1994
Movie : Promised Land | id : tt2091473 | score : 0.1938
Movie : Passion Fish | id : tt0105107 | score : 0.1906
Movie : Outside In | id : tt7260048 | score : 0.1899


In [298]:
def titre_index(titre: str):
    return df[df.titre_str == titre].index[0]

def idx_titre(idx: int):
    return df[df.index == idx]["titre_str"].values[0]

def idx_titre_id(idx: int):
    return df[df.index == idx]["titre_id"].values[0]

# def idx_title(idx):
#     return df[df.index == idx]["titre_str"].values[0]

In [299]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
count_matrix = cv.fit_transform(df['one_for_all'])
movie_user_likes = "Fight Club"
movie_index = titre_index(movie_user_likes)

knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(count_matrix)
dist, indices = knn_model.kneighbors(count_matrix[movie_index], n_neighbors=15)

print(f"Top 10 similar movies to {movie_user_likes} are :\n")
for index, dis in zip(indices.flatten()[1:], dist.flatten()[1:]):
    print(f"Movie : {idx_title(index)} | id : {idx_titre_id(index)} | score : {np.round(dis, 4)}")


Top 10 similar movies to Fight Club are :

Movie : Never Back Down | id : tt1023111 | score : 0.8202
Movie : The Jane Austen Book Club | id : tt0866437 | score : 0.849
Movie : Run Hide Fight | id : tt11456054 | score : 0.8608
Movie : Exotica | id : tt0109759 | score : 0.861
Movie : Breakfast at Tiffany's | id : tt0054698 | score : 0.8673
Movie : Kafka | id : tt0102181 | score : 0.8706
Movie : The Naked Kiss | id : tt0058390 | score : 0.8728
Movie : Gangs of New York | id : tt0217505 | score : 0.8743
Movie : This Is England | id : tt0480025 | score : 0.8746
Movie : Incident in a Ghostland | id : tt6195094 | score : 0.8779
Movie : After Lucia | id : tt2368749 | score : 0.878
Movie : Project Almanac | id : tt2436386 | score : 0.878
Movie : Racing with the Moon | id : tt0087968 | score : 0.8788
Movie : The Painted Veil | id : tt0446755 | score : 0.8804


In [300]:

# random_model = RandomForestClassifier().fit(combined_matrix, df["titre_genres"])
# for idx, movies in zip(sim_mov_idx, same_movies):
#     mo = combined_matrix[idx].toarray()
#     prediction = random_model.predict(mo)[0]
#     confidence = np.max(random_model.predict_proba(mo))

#     print("movie :", movies)
#     print("confidence :", confidence)

In [301]:
# Movie : Avengers: Age of Ultron | id : tt2395427 | score : 1.0
# Movie : The Gray Man | id : tt1649418 | score : 0.2522
# Movie : Avengers: Endgame | id : tt4154796 | score : 0.1226
# Movie : Snowpiercer | id : tt1706620 | score : 0.1209
# Movie : Kingsman: The Secret Service | id : tt2802144 | score : 0.1117
# print(f"Top 10 similar movies to {movie_user_likes} are :\n")
#

In [302]:
# poids_ = {
#     "titre_str": 0.1,
#     "titre_genres": 0.1,
#     "directors": 0.2,
#     "overview": 0.2,
#     "actor_1": 0.15,
#     "actor_2": 0.1,
    # "actor_3": 0.025,
#     "actor_4": 0.025,
# }

# full_matrix = []
# for col, poids in poids_.items():
#     tfidf_ = TfidfVectorizer()
#     matrix_ = tfidf_.fit_transform(df[col]) * poids
#     full_matrix.append(matrix_)

# combined_matrix = hstack(full_matrix)

# cosine = cosine_similarity(combined_matrix)

# movies = movie
# mov_idx = title_index(movies)
# similar = cosine[mov_idx]
# similar1 = list(enumerate(cosine[mov_idx]))
# sim_scores = sorted(similar1, key=lambda x: x[1], reverse=True)

# sim_mov_idx = similar.argsort()[::-1][1:11]
# same_movies = df.loc[sim_mov_idx, "titre_str"]
# sim_scores[1:11]
# score = [i[1] for i in sim_scores]
# for movies, score in zip(same_movies, score):
#     print(movies, score)


# # random_model = RandomForestClassifier().fit(matrix, df["titre_genres"])
# # for idx, movies in zip(sim_mov_idx, same_movies):
# #     mo = matrix[idx].toarray()
# #     prediction = random_model.predict(mo)[0]
# #     confidence = np.max(random_model.predict_proba(mo))

# #     print("movie :", movies)
# #     print("confidence :", confidence)